In [ ]:

import cv2
import torch
from ultralytics import YOLO
import os
import shutil
from PIL import Image

plate_detector = YOLO("/Users/PhaPhan/Downloads/Stage 1/run_l/exp5/weights/best.pt")
char_recognizer = YOLO("/Users/PhaPhan/Downloads/Stage 2/run_l/exp2/weights/best.pt")

def check(s):
    if len(s) == 6 and s[0].isalpha() and s[1].isalpha():
        return 0
    if "NN" in s and len(s) in {8, 9, 10}:
        return 0
    if len(s) not in {8, 9}:
        return 1
    if not s[2].isalpha():
        return 1
    if any(not c.isdigit() for i, c in enumerate(s) if i != 2 and i != 3):
        return 1
    return 0

def expand_to_640(image):
    height, width = image.shape[:2]
    new_width = 2560
    new_height = int(height * (2560 / width))
    return cv2.resize(image, (new_width, new_height))

def recognize_plate(image_path, output_folder):
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not load image")
        return

    os.makedirs(output_folder, exist_ok=True)

    plate_results = plate_detector(image)
    plate_count = 0

    for result in plate_results:
        for box in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            plate_region = image[y1:y2, x1:x2].copy()
            
            expanded_plate = expand_to_640(plate_region)
            plate_folder = os.path.join(output_folder, f"plate_{plate_count}")
            os.makedirs(plate_folder, exist_ok=True)
            expanded_path = os.path.join(plate_folder, "expanded.jpg")
            cv2.imwrite(expanded_path, expanded_plate)
            
            char_results = char_recognizer.predict(expanded_path, save=True, conf=0.5, project=plate_folder, name="yolo_output")
            result_dir = char_results[0].save_dir            

            plate_text = ""
            char_dict = {}
            for char_result in char_results:
                for idx, char_box in enumerate(char_result.boxes.xyxy):
                    x1c, y1c, x2c, y2c = map(int, char_box)
                    cls_index = int(char_result.boxes.cls[idx])
                    conf_score = float(char_result.boxes.conf[idx])
                    char_label = char_result.names[cls_index]

                    if int(char_label) < 9:
                        char_label = chr(int(char_label) + 49)
                    elif int(char_label) < 17:
                        char_label = chr(int(char_label) + 56)
                    elif int(char_label) < 21:
                        char_label = chr(int(char_label) + 58)
                    elif int(char_label) == 21:
                        char_label = chr(int(char_label) + 59)
                    elif int(char_label) < 26:
                        char_label = chr(int(char_label) + 61)
                    elif int(char_label) < 29:
                        char_label = chr(int(char_label) + 62)
                    else:
                        char_label = chr(48)

                    box_height = (y2c - y1c)*1.5
                    font_scale = box_height / 80
                    font_thickness = max(1, int(box_height / 50)) 
                    label = f"{char_label}"                        

                    (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)

                    text_bg_topleft = (x1c, y1c - text_height - 6)
                    text_bg_bottomright = (x1c + text_width + 6, y1c)                                                       

                    cv2.rectangle(expanded_plate, text_bg_topleft, text_bg_bottomright, (16, 12, 23), -1)                
                    cv2.rectangle(expanded_plate, (x1c, y1c), (x2c, y2c), (16, 12, 23), 2)
                    cv2.putText(expanded_plate, label, (x1c + 3, y1c - 6), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (175, 154, 247), font_thickness)

                    char_dict[int(idx)] = (char_label, x1c, y1c, x2c, y2c)
            
            expanded_image_path = os.path.join(plate_folder, "expanded_with_annotations.jpg")
            cv2.imwrite(expanded_image_path, expanded_plate)
            
            char_items = list(char_dict.items())
            char_items = sorted(char_dict.items(), key=lambda item: item[1][1])

            i = 0
            front = 0
            flip = 0

            while i < len(char_items) - 1:
                idx1, (char1, cx1, cy1, mx1, my1) = char_items[i]
                idx2, (char2, cx2, cy2, mx2, my2) = char_items[i + 1]

                #print(char1)
                #print(char_items)        
                #print('front =', front)        

                if (my1) - (cy2) < (3 / 10) * (my1 - cy1):
                    if flip == char_items[i + 1]:
                        break
                    if flip == 0:
                        flip = char_items[i + 1]
                    char_items.append(char_items.pop(i + 1))                     
                    front = i+1                              
                    i-=1          
                elif (my2) - (cy1) < (3 / 10) * (my2 - cy2):
                    char_items.insert(front, char_items.pop(i + 1))
                    #char_items.append(char_items.pop(i+1))
                    i = front - 1
                    front += 1                    
                i += 1            

            plate_text = "".join([char for _, (char, _, _, _, _) in char_items])

            if check(plate_text) == 0:                
                cv2.rectangle(image, (x1, y1), (x2, y2), (16, 12, 23), 2)
                text_size = cv2.getTextSize(plate_text, cv2.FONT_HERSHEY_SIMPLEX, 2, 2)[0]
                cv2.rectangle(image, (x1, y1 - text_size[1] - 10), (x1 + text_size[0], y1), (16, 12, 23), -1)
                cv2.putText(image, plate_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (175,154,247), 2)

            plate_count += 1    
    cv2.imwrite(os.path.join(output_folder, "final_output.jpg"), image)

recognize_plate("/Users/PhaPhan/Downloads/Image.jpg", "/Users/PhaPhan/Downloads/0.Output/Image")


0: 224x640 7 0s, 343.4ms
Speed: 3.5ms preprocess, 343.4ms inference, 0.9ms postprocess per image at shape (1, 3, 224, 640)

image 1/1 /Users/PhaPhan/Downloads/0.Output/Image13/plate_0/expanded.jpg: 288x640 1 0, 3 1s, 3 2s, 4 3s, 2 4s, 1 5, 1 18, 1 22, 1 29, 376.0ms
Speed: 3.1ms preprocess, 376.0ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 640)
Results saved to /Users/PhaPhan/Downloads/0.Output/Image13/plate_0/yolo_output

image 1/1 /Users/PhaPhan/Downloads/0.Output/Image13/plate_1/expanded.jpg: 320x640 2 0s, 2 1s, 5 4s, 1 5, 1 6, 4 8s, 1 13, 1 23, 1 29, 406.3ms
Speed: 2.5ms preprocess, 406.3ms inference, 0.8ms postprocess per image at shape (1, 3, 320, 640)
Results saved to /Users/PhaPhan/Downloads/0.Output/Image13/plate_1/yolo_output

image 1/1 /Users/PhaPhan/Downloads/0.Output/Image13/plate_2/expanded.jpg: 480x640 1 0, 1 1, 3 4s, 1 5, 2 8s, 1 13, 1 29, 611.5ms
Speed: 4.7ms preprocess, 611.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
Results 